<a href="https://colab.research.google.com/github/Fernando2528/Programacion.Aplicada-Fernando-Andres-Rojas-Fernandez-/blob/main/Servidor_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Note: in tis file will contain the code of a server that allows the comunication between the fornt end(HTML) and the Back end(Circuito Python) having as purpose to allow the remote control of the mechanical arm.

##Version 0.1 (test version)

###Main Objetive: Manage the led status trought remote indications. (Remote inputs)

In [ ]:
import wifi
import socketpool
import board
import time
import digitalio

# Conectar a la red Wi-Fi
wifi.radio.connect("PruebaPi", "11223344")
pool = socketpool.SocketPool(wifi.radio)

# Mostrar la dirección IP asignada
print("Connected to Wi-Fi")
print("IP Address:", wifi.radio.ipv4_address)

# Configurar el LED
led = digitalio.DigitalInOut(board.GP14)
led.direction = digitalio.Direction.OUTPUT

# Configurar el servidor para redirigir al navegador
PORT = 80
s = pool.socket()
s.setsockopt(pool.SOL_SOCKET, pool.SO_REUSEADDR, 1)
s.bind(("", PORT))
s.listen(5)

print("Serving at port", PORT)

while True:
    conn, addr = s.accept()
    print("Got a connection from", addr)

    # Leer datos de la conexión
    try:
        request = conn.recv(1024)
        request_str = str(request, 'utf-8')
        print("Request:", request_str)

        # Control del LED basado en la solicitud
        if "Servomotor_1_Led_1_Status=true" in request_str:
            led.value = True
            print("LED encendido")
        elif "Servomotor_1_Led_2_Status=true" in request_str:
            led.value = False
            print("LED apagado")

    except Exception as e:
        print("Error processing request:", e)

    # Enviar la redirección HTTP al navegador
    github_html_url = "https://fernando2528.github.io/Programacion.Aplicada-Fernando-Andres-Rojas-Fernandez-/Remote_Control.html"
    try:
        conn.send(b"HTTP/1.1 302 Found\r\n")
        conn.send(f"Location: {github_html_url}\r\n".encode('utf-8'))
        conn.send(b"Connection: close\r\n\r\n")
    except Exception as e:
        print("Error sending response:", e)
    finally:
        conn.close()

    # Pequeña pausa para evitar saturar el loop
    time.sleep(0.1)
